[py060 Простая отладка сетевых приложений](#py060-Простая-отладка-сетевых-приложений)   
[py061 Сетевой вариант крестиков-ноликов на Питоне](#py061-Сетевой-вариант-крестиков-ноликов-на-Питоне)   
[py062 элементы тестирования консольной программы](#py062-элементы-тестирования-консольной-программы)   
[py063 игра Змейка](#py063-игра-Змейка)   
[py064 Python отладчик для исправления ошибок](#py064-Python-отладчик-для-исправления-ошибок)   
[py065 Python работа с сетью на канальном уровне](#py065-Python-работа-с-сетью-на-канальном-уровне)   
[py066 Пятнашки на Питоне](#py066-Пятнашки-на-Питоне)   
[py067 Питон программа управления ШИМ контроллера](#py067-Питон-программа-управления-ШИМ-контроллера)   
[py068 Питон скачиваем документы по ссылкам](#py068-Питон-скачиваем-документы-по-ссылкам)   
[py069 Формируем HTML тест из DOC файла](#py069-Формируем-HTML-тест-из-DOC-файла)   


#### py060 Простая отладка сетевых приложений
[teamviewer-portable.ru](https://teamviewer-portable.ru.uptodown.com/windows)

#### py061 Сетевой вариант крестиков-ноликов на Питоне
#### py062 элементы тестирования консольной программы

In [ ]:
##### python py62tst.py py62ex.py py62in.txt > py62out.txt

In [2]:
%%writefile py62ex.py
#== 01.py 
def fn(st):
    if st[0] == 'm':
        return st*3
    else:
        return st*2

while 1:
    st = input(':')
    if st == 'x': 
        break 
    st = fn(st)
    print (st)

print ('end of work')


Writing py62ex.py


In [5]:
%%writefile py62in.txt
#== in01.txt  -  test for 01.py
05
26
medved
x


Writing py62in.txt


py62out.txt
```
0505
2626
medvedmedvedmedved
end of work
```

In [7]:
%%writefile py62tst.py
#== test.py   by A.B.Glazov
#== test work of consol python programm
import sys

#-- handle run error 
if len(sys.argv) <  3:
    print ('error of usage')
    print ('you must write:')
    print ('    test.py consol.py inp.txt > out.txt')
    print ('        where:')
    print ('    consol.py - name of testing module')
    print ('    inp.txt - file with the list of input strings')
    print ('    out.txt - file for writing all answers')
    sys.exit()

testing_file = sys.argv[1]
input_file   = sys.argv[2]

#-- prepare list of input strings
fh_input = open(input_file)
ls_input = fh_input.readlines()
fh_input.close()

num = len(ls_input) - 1
while  num >=0:
    if ls_input[num][0] == '#':   #-- delete comments
        res = ls_input.pop(num) 
    else:                         #-- cut end of line     
        ls_input[num] = ls_input[num].strip()
    num -= 1

#-- redefine raw_input    
in_num = 0
def input(param):
    global in_num
    in_st = ls_input[in_num]
    in_num +=1
    return in_st

#-- handle testing module
# res = execfile(testing_file)
res = exec(compile(open(testing_file).read(), testing_file, 'exec'))

Overwriting py62tst.py


#### py063 игра Змейка

In [11]:
# coding: utf-8
#-- игра змейка.  А.Б.Глазов
##-- цель - управляя змейкой вырастить ее как можно больше
##-- Клавиши управления:
##-- <Enter>- начать новую игру
##-- стрелки "<- ", "->" поворачивают змейку влево и вправо
##-- клавиши "q ", "s"  соответственно ускоряют и замедляют движение змейки
##-- клавиши "a ", "r"  включают и отключают мозги))) змейки
##-- клавиша "Пробел"  останавливает и продолжает игру

from tkinter import *
from random import randint
root = Tk()

#-- параметры поля
cell_size = 24
col_count = 30
row_count = 20
col_middle = col_count//2
row_middle = row_count//2

#-- глобальные переменные
game_over = 0
food = 0
snake = []
foods = []
food_time = randint(5,10)
time_interval = 400
mode = "manual"
stop_flag = True

#-- список напрвлений движения и номер выбранного напрвления
ls_dir = [(0, 1),( 1,0),(0, -1),(-1,0)]
num_dir = 3;

#-- массив результатов движения
cells = []
for row_num in range(row_count):
    row = []
    for col_num in range(col_count):
        row.append(0)
    cells.append(row)

#-- формирование канвы
canv_width = col_count * cell_size
canv_height = row_count * cell_size
canv = Canvas(root, width = canv_width, height = canv_height, bg = "darkgreen")
border = canv.create_rectangle(2,2, canv_width, canv_height, width = 3,outline = "black")
canv.pack()

#-- нарисовать звено змейки
def draw_part(rcp):
    (row_num, col_num, part_name) = rcp 
    x = col_num * cell_size
    y = row_num * cell_size
    canv.create_rectangle(x, y, x + cell_size, y + cell_size,  fill = "yellow", width = 2, outline = "black",   tag = part_name)

#-- сдвинуть голову змеи
def move_head():
    global snake
    global food
    global game_over
    global foods
    global mode
    global num_dir

    row0, col0, tag0 = snake[0]

    #== если ражим auto, то скорректировать направление
    if mode == "auto":
        #-- рассчитаеть предполагаемое следующее значение
        d_row, d_col = ls_dir[ num_dir ]
        row = row0 + d_row
        col = col0 + d_col
        if row < 0 or row >= row_count or  col < 0 or col >= col_count:
            val_next = -1
        else:
            val_next = cells[row][col]

        #-- проверить соседние значения на больше следующего
        for num in range(4):
            d_row, d_col = ls_dir[ num ]
            row = row0 + d_row
            col = col0 + d_col
            if row < 0 or row >= row_count or  col < 0 or col >= col_count:
                val = -1
            else:
                val = cells[row][col]
            if val > val_next:
                num_dir = num
                val_next = val

    #-- рассчитать следующую позицию
    d_row, d_col = ls_dir[ num_dir ]
    row = row0 + d_row
    col = col0 + d_col
    row = int(row)
    col = int(col)

    #-- сформировать новое звено,если ячейка пуста
    if row >= 0 and row < row_count and  col >= 0 and col < col_count:
        tag_num = int(tag0[1:]) + 1
        tag = "t"+str(tag_num)

        snake_part = (row, col, tag)	
        snake.insert(0,snake_part)
        if cells[row][col] < 0:
            game_over = 1
            return

        if cells[row][ col ] > 0:
            food += cells[row][ col ]
            food_tag = "p"+str( row * col_count + col)
            canv.delete(food_tag)

        draw_part((row, col, tag))	
        cells[row][col ] = -1	#-- признак занятости ячейки
    else:
        game_over = 1

#-- сдвинуть хвост змеи
def move_tail():
    global snake
    global food

    if food > 0:
        food -= 1
    elif game_over:
        pass
    else:	
        row, col, tag = snake[-1]
        cells[row][ col ] = 0
        res = snake.pop(-1)
        canv.delete(tag)

#-- создать корзинку с яблоками или капкан
def create_food():
    global foods
    #-- случайное значение (капкан - отрицательное, иначе число яблок в корзине)
    food_val = randint(-2,8)
    if food_val == 0:
        return 0
    #-- цвет капкана синий, продуктов-красный
    food_color = "red"
    if food_val < 0:
        food_val -= 1
        food_color = "blue"

    row = randint(0,row_count-1)
    col = randint(0,col_count-1)
    #-- ставим только в пустую ячейку
    if cells[ row][col] == 0:
        cells[ row][col] = food_val
        #-- тэг продукта определяется его местом
        food_tag = "p"+str( row * col_count + col)
        #-- добавить в список продуктов
        foods.append( ( row, col, food_tag, food_val ) )
        #-- нарисовать продукт на канве
        x = col * cell_size
        y = row * cell_size
        canv.create_oval(x+1, y+1, x + cell_size-2, y + cell_size-2, fill = food_color, width = 1, outline = "white", tag = food_tag )

    return food_val

#-- обработчик клавиатуры
def key_handl(event):
    # обработать нажатия клавиш
    global game_over
    global cells
    global num_dir
    global snake
    global tag_num
    global foods
    global food_time
    global time_interval
    global mode
    global stop_flag

    #print event.keysym

    if event.keysym == "space" :
        stop_flag = not stop_flag


    if event.keysym == "q" :
        time_interval =  int(time_interval /1.25)

    if event.keysym == "s" :
        time_interval =  int(time_interval *1.25)

    if event.keysym == "a" :
        mode = "auto"

    if event.keysym == "r" :
        mode = "manual"

    if event.keysym == "Left":
        num_dir -= 1
        if num_dir < 0:
            num_dir = len(ls_dir) - 1

    elif event.keysym == "Right":
        num_dir += 1
        if num_dir >= len(ls_dir):
            num_dir = 0

    #== начать игру заново по клавише  Enter
    elif event.keysym == "Return" and game_over:
        game_over = 0
        stop_flag = True

        #-- очистить ячейки массива
        for row in range(row_count):
            for col in range(col_count):
                cells[row][col] = 0

        #-- стереть ячейки канвы со змейкой
        for (row, col, snake_tag) in snake:
            canv.delete(snake_tag)		

        #-- стереть ячейки канвы с яблоками
        for (row, col, food_tag, val) in foods:
            canv.delete(food_tag)
        foods = []
        food_time = randint(5,10)

        canv.delete("t0")
        canv.delete("game")

        #-- нарисовать начальную змейку из двух звеньев
        snake = [(row_middle,col_middle,"t1"),(row_middle + 1,col_middle,"t0")]
        for snake_part in snake:
            draw_part(snake_part)

#-- подключить обработчик клавиатуры
canv.bind("<KeyPress>", key_handl)
canv.focus_set()
#-- нарисовать начальную змейку из двух звеньев
snake = [(row_middle,col_middle,"t1"),(row_middle + 1,col_middle,"t0")]
for snake_part in snake:
    draw_part(snake_part)


#-- главная функция, запускаемая в цикле

def main():
    global cells
    global snake
    global foods
    global food_time
    global time_interval
    if game_over == 0 and not stop_flag:
        move_head()	
        move_tail()

        #--- обработать вставку продукта
        if food_time > 0:
            food_time -=1
        else:
            if create_food():
                food_time = randint(5,10)

    elif game_over == 1:
        st = "game over, len of snake = " + str(len(snake))
        canv.create_text(180, 30, text = st, font = ("Arial", 20), fill = "white", tag = "game")

    root.after(time_interval, main)

main()

root.mainloop()


In [5]:
k = (1,4)
def foo(a_b):
    (a,b)=a_b
    return a+b
foo((1,3))

4

#### py064 Python отладчик для исправления ошибок

[https://github.com/pyscripter/pyscripter](https://github.com/pyscripter/pyscripter)  
[]()  
C:\Program Files\PyScripter  
`Run` `Debug F9`  
`F8` `F7`  
`View` `Navigate` `Watch Ctrl-Alt-W`

In [ ]:
# previouse variant
def ins_frel(frels, frel_num, sub):
        frels.insert( frel_num, sub )

In [1]:
#== coding: utf-8
#== str_hndl.py         A.B.Glazov
#== простая программа обработки строк

#== преобразовать строку в список подстрок
#== пример     frels = srt2frel("preved|medved")
def str2frel(st):
        ls = st.split('|')
        ls.insert(0,"0")
        return ls

#== преобразовать список подстрок в строку
#== пример     st = frel2str(frels)
def frel2str(frels):
        frels.pop(0)
        st = "|".join(frels)
        return st

#== получить число подстрок в строке
#== пример     frels_count = count_frels(frels)
def count_frels(frels):
        return len(frels)-1

#== добавить подстроку в  список подстрок
#== пример     add_frel(frels, sub)
def add_frel(frels, sub):
        frels.append( sub )

#== удалить подстроку из списка подстрок
#== пример     del_frel(frels, frel_num)
def del_frel(frels, frel_num):
        frels.pop( frel_num )


#== установить новое значение подстроки в списке подстрок
#== пример     set_frel(frels, frel_num, sub)
def set_frel(frels, frel_num, sub):
        frels[ frel_num ] = sub

#== вставить новую подстроку в списке подстрок
#== пример     ins_frel(frels, frel_num, sub)
def ins_frel(frels, frel_num, sub):
    if frel_num>0:
        frels.insert( frel_num, sub )
    else:
        print("index out of range")

print (u"Программа обработки строк")

st = input(u'введите строку с разделителями "|"\n: ')

while(1):
        print (u"\nстрока: ", st)
        frels = str2frel( st )
        frels_count = count_frels( frels )
        print (u"число подстрок: ", frels_count)
        print (u"\nвозможные команды:\n add:SSS, set:NNN:SSS, ins:NNN:SSS, del:NNN, exit")
        print (u"где     SSS - произвольная строка,   NNN - номер подстроки ")

        cmd = input(u'введите команду: ')
        ar_cmd = cmd.split(":")
        if ar_cmd[0] == 'exit':
                break
        elif ar_cmd[0] == 'add':
                sub = ar_cmd[1]
                add_frel(frels, sub)
                st = frel2str(frels)

        elif ar_cmd[0] == 'set':
                frel_num = int( ar_cmd[1] )
                sub = ar_cmd[2]
                set_frel(frels, frel_num, sub)
                st = frel2str(frels)

        elif ar_cmd[0] == 'ins':
                frel_num = int( ar_cmd[1] )
                sub = ar_cmd[2]
                ins_frel(frels, frel_num, sub)
                st = frel2str(frels)

        elif ar_cmd[0] == 'del':
                frel_num = int( ar_cmd[1] )
                del_frel( frels, frel_num )
                st = frel2str(frels)

print ('OK')


Программа обработки строк
введите строку с разделителями "|"
: 1|3

строка:  1|3
число подстрок:  2

возможные команды:
 add:SSS, set:NNN:SSS, ins:NNN:SSS, del:NNN, exit
где     SSS - произвольная строка,   NNN - номер подстроки 
введите команду: ins:0:1
index out of range

строка:  1|3
число подстрок:  2

возможные команды:
 add:SSS, set:NNN:SSS, ins:NNN:SSS, del:NNN, exit
где     SSS - произвольная строка,   NNN - номер подстроки 
введите команду: exit
OK


#### py065 Python работа с сетью на канальном уровне

[https://www.winpcap.org/](https://www.winpcap.org/)  
pip install winpcapy  
Successfully installed winpcapy-1.0.2  
[https://pypi.org/project/WinPcapy/](https://pypi.org/project/WinPcapy/)  


In [11]:
%%writefile py65pcapy.py
# Easy Packet live callback
from winpcapy import WinPcapUtils

# Example Callback function to parse IP packets
def packet_callback(win_pcap, param, header, pkt_data):
    # Assuming IP (for real parsing use modules like dpkt)
    ip_frame = pkt_data[14:]
    # Parse ips
    src_ip = ".".join([str(b) for b in ip_frame[0xc:0x10]])
    dst_ip = ".".join([str(b) for b in ip_frame[0x10:0x14]])
    print("%s -> %s" % (src_ip, dst_ip))
#     print('aaa')

WinPcapUtils.capture_on("*Ethernet*", packet_callback)

Overwriting py65pcapy.py


D:\mailCloud\prjother\089hosts\jnotebook\pyoop\prkt>python py65pcapy.py
Traceback (most recent call last):
  File "_ctypes/callbacks.c", line 232, in 'calling callback function'
  File "C:\ProgramData\Anaconda3\lib\site-packages\winpcapy\winpcapy.py", line 1
13, in packet_handler
    return self._callback(self, param, header, pkt_data)
  File "py65pcapy.py", line 9, in packet_callback
    src_ip = ".".join([str(ord(b)) for b in ip_frame[0xc:0x10]])
  File "py65pcapy.py", line 9, in <listcomp>
    src_ip = ".".join([str(ord(b)) for b in ip_frame[0xc:0x10]])
TypeError: ord() expected string of length 1, but int found
Traceback (most recent call last):
  File "_ctypes/callbacks.c", line 232, in 'calling callback function'
  File "C:\ProgramData\Anaconda3\lib\site-packages\winpcapy\winpcapy.py", line 1
13, in packet_handler
    return self._callback(self, param, header, pkt_data)
  File "py65pcapy.py", line 9, in packet_callback
    src_ip = ".".join([str(ord(b)) for b in ip_frame[0xc:0x10]])
  File "py65pcapy.py", line 9, in <listcomp>
    src_ip = ".".join([str(ord(b)) for b in ip_frame[0xc:0x10]])
TypeError: ord() expected string of length 1, but int found
Traceback (most recent call last):
  File "_ctypes/callbacks.c", line 232, in 'calling callback function'
  File "C:\ProgramData\Anaconda3\lib\site-packages\winpcapy\winpcapy.py", line 1

In [8]:
print(ord(str(1)))

49


#### py066 Пятнашки на Питоне

In [1]:
# coding: utf-8
#-- игра пятнашки.  А.Б.Глазов
import random
from tkinter import *

#-- переменные
##-- параметры поля
cell_size = 60
x_count = 4
y_count = 4

##-- признак окончания игры
game_over = -1

##-- массив игровых элементов
elems = ["e1","e2","e3","e4","e5","e6","e7","e8",
    "e9","e10","e11","e12","e13","e14","e15","e0"]

##-- массив цветов элементов
colors = ["black","silver","green","lightgreen",
            "yellow","orange","red","deepskyblue",
            "rosybrown","lawngreen","pink", "palegreen",
            "tan","lightblue", "lime","violet"]
#-- функции
##-- создать и отрисовать элемент на  канве
def create_elem( elem_num):
    el = elems[elem_num]
    row_num = elem_num // 4
    col_num = elem_num % 4
    color_num = int(el[1:])
    elem_color = colors[color_num]
    x_left = col_num * cell_size
    y_top = row_num * cell_size
    canv.create_rectangle(x_left + 1, y_top + 1, x_left + cell_size - 2, y_top + cell_size - 2,
            fill = elem_color, outline="#050", width = 2, tag = el)
    canv.create_text(x_left + 30, y_top + 30, text = el[1:], font = ("Arial", 20), tag = "t"+el)

##-- просто отрисовать элемент на  канве
def draw_elem( elem_num):
    el = elems[elem_num]
    row_num = elem_num // 4
    col_num = elem_num % 4
    color_num = int(el[1:])
    elem_color = colors[color_num]
    x_left = col_num * cell_size
    y_top = row_num * cell_size
    canv.coords(el, x_left + 1, y_top + 1 , x_left + cell_size - 2, y_top + cell_size - 2 )
    canv.coords("t"+el, x_left + 30, y_top + 30 )

##-- поменять местами 2 элемента
def change_elems( elem_num1, elem_num2 ):
    elems[elem_num1], elems[elem_num2] = elems[elem_num2], elems[elem_num1]
    draw_elem( elem_num1 )
    draw_elem( elem_num2 )

##-- проверить на завершение
def test_win( ):
    for elem_num in range(15):
        el = elems[elem_num]
        if elem_num + 1 != int(el[1:]):
            return 0
    return 1

##-- получить список соседних ячеек для элемента массива
def get_near(elem_num):
    near_cells = []
    row_num = elem_num // 4
    col_num = elem_num % 4
    print(row_num,col_num)
    if col_num > 0:
        tmp_num = col_num + row_num * 4 - 1
        near_cells.append(tmp_num)
    if col_num < 3:
        tmp_num = col_num + row_num * 4 + 1
        near_cells.append(tmp_num)
    if row_num > 0:
        tmp_num = col_num + (row_num -1) * 4
        near_cells.append(tmp_num)
    if row_num < 3:
        tmp_num = col_num + (row_num + 1) * 4
        near_cells.append(tmp_num)
    return near_cells	

##-- подготовить список элементов
def prep_elems(level):
    global elems

    if level == "Return":
        random.shuffle(elems)
    else:
        try:
            shift_count = int(level) + 1
        except:
            shift_count = 10

        shift_count *= 3
        elem_num = elems.index("e0")
        for shift_num in range(shift_count):
            near_cells = get_near(elem_num)
            random.shuffle( near_cells )
            tmp_num = near_cells[0]
            change_elems( elem_num, tmp_num )
            elem_num = tmp_num

#-- графическая часть игры
##-- формирование канвы
canv_width = x_count * cell_size
canv_height = y_count * cell_size

root = Tk()
canv = Canvas(root, width = canv_width, height = canv_height)
canv.pack()


##-- клеточки на канве
for x_num in range(x_count):
    for y_num in range(y_count):
        canv.create_line(0, y_num*cell_size, canv_width, y_num*cell_size)
        canv.create_line( x_num*cell_size, 0, x_num*cell_size, canv_height)

##-- текст при входе в игру
st = "  GAME 15 \n  select level(0-9)\n  or press <Enter>"
canv.create_text(120, 120, text = st, font = ("Arial", 20), tag ="txt"  )

#-- обработчики событий
##-- сдвинуть при возможности элементы
def test_elems(event):
    global game_over
    if game_over:
        return
    #-- определить выбранный номер элемента
    col_num = event.x // cell_size
    row_num = event.y // cell_size
    elem_num = col_num + row_num * 4

    el = elems[elem_num]
    #-- отработать щелчок по пустому полю
    if el == "e0":
        return

    #-- определить ячейки-соседи
    near_cells = get_near(elem_num)

    #-- проверить ячейки-соседи на 0
    for tmp_num in near_cells:
        if elems[tmp_num] == "e0":
            change_elems( elem_num, tmp_num )
            game_over = test_win()
            if game_over:
                st = "GAME OVER"
                canv.create_text(120, 120, text = st, font = ("Arial", 20), tag ="txt"  )		
    return			

##-- привязка к событиям мыши
canv.bind("<Button-1>", test_elems)

##-- обработать нажатия клавиш
def key_handl(event):
    global game_over
    symbols = ["0","1","2","3","4","5","6","7","8","9","Return"]
    if event.keysym in symbols and game_over != 0:
        canv.delete("txt")	
        level = event.keysym
        prep_elems(level)

        if game_over == -1 :  # начать игру в первый раз
            for elem_num in range(len(elems)):
                create_elem(elem_num)
        elif game_over == 1:  # начать игру заново
            for elem_num in range(len(elems)):
                draw_elem(elem_num)
        game_over = 0

##-- привязка к событиям клавиатуры
canv.bind("<KeyPress>", key_handl)
canv.focus_set()

#-- запустить игру
root.mainloop()


2 2
2 3


#### py067 Питон программа управления ШИМ контроллера

#### py068 Питон скачиваем документы по ссылкам

In [39]:
from urllib.request import urlopen, urlretrieve, unquote, quote
from urllib import parse
import urllib
s="http://ef-pmr.org/uploads/law/121_zakon_n121-З-III_29-04-2002.rar"
bytes=s.encode()
urllib.parse.quote_from_bytes(bytes, safe=':/')

'http://ef-pmr.org/uploads/law/121_zakon_n121-%D0%97-III_29-04-2002.rar'

[egass](http://ef-pmr.org/law)  

In [6]:
!mkdir efpmr

In [41]:
#скачать все документы из сайта "http://ef-pmr.org/law/"
#предварительно слздать каталог 'efpmr'
#и скопировать в него unrar.exe из "program files\winrar"

#перейти в подкаталог для скачивания файлов
import os
# os.chdir('efpmr')
print (os.getcwd())

#сформировать список всех ссылок
from urllib.request import urlopen, urlretrieve, unquote
from urllib import parse
import urllib
from bs4 import BeautifulSoup

url = "http://ef-pmr.org/law/"
      
content = urlopen( url )
page2 = content.read()
page = page2.decode('utf-8')


soup = BeautifulSoup(page)

# print(soup.prettify())

body = soup.find('body')
ref_list = body.findAll('a')

#выбрать все ссылки на документы
filref_list = []
for num in range(len(ref_list)):
    ref = ref_list[num]
    href = ref['href']
    if href[-4:] in ['docx','.pdf','.rar','.zip','.doc']:
        filref_list.append(href)

#убрать возможное дублирование
print (len(filref_list))
setref_list = set(filref_list)
filref_list = list(setref_list)
print (len(filref_list))

#функция формирования имени файла из ссылки 
def form_filename2(href):
    print(href)
    href1 = href.encode('utf-8').decode('utf-8')
    href2 = unquote(href1)
#     href3 = href2.decode('utf-8').encode('cp1251')
    href3 = href1.replace(' ','%20')
    print(href3)
    file_name = href3.split('/')[-1]
    return href3, file_name

def form_filename(href):
    print(href)
    href1 = href.encode('utf-8').decode('utf-8')

#     s="http://ef-pmr.org/uploads/law/121_zakon_n121-З-III_29-04-2002.rar"
    href2=href1.encode()
    href3 = urllib.parse.quote_from_bytes(href2, safe=':/')    
    
    
#     href2 = unquote(href1)
#     href3 = href2.decode('utf-8').encode('cp1251')
#     href3 = href1.replace(' ','%20')
    print(href3)
    file_name = href3.split('/')[-1]
    return href3, file_name

#Простое скачивание:
docref_list = [href for href in filref_list \
     if href[-4:] in ['docx','.doc','.pdf']]
print (len(docref_list))
    
for href in docref_list:
    href1, file_name = form_filename(href)
    print (file_name)
    res = urlretrieve(href1, file_name)

#Работа с RAR архивами            
rarref_list = [href for href in filref_list if href[-4:] in ['.rar']]
print (len(rarref_list))

for href in rarref_list:
    href1, file_name = form_filename(href)
    print (file_name)
    res = urlretrieve(href1, file_name)
    res = os.system('unrar.exe e '+ file_name)
    os.remove( file_name )

#Работа с ZIP архивами    
import zipfile
    
zipref_list = [href for href in filref_list if href[-4:] in ['.zip']]
print (len(zipref_list))
    
for href in zipref_list:
    href1, file_name = form_filename(href)
    print (file_name)
    res = urlretrieve(href1, file_name)
    zip_ref = zipfile.ZipFile(file_name, 'r')
    for dos_name in zip_ref.namelist():
        full_dosname = zip_ref.extract(dos_name)
        str2 = dos_name.encode('cp437')
        win_name = str2.decode('cp866')
        os.rename(dos_name, win_name)
    zip_ref.close()
    os.remove( file_name )


D:\mailCloud\prjother\089hosts\jnotebook\pyoop\prkt\efpmr
200
100
47
http://ef-pmr.org/uploads/law/100_postanovlenie_n50_13-03-2013.docx
http://ef-pmr.org/uploads/law/100_postanovlenie_n50_13-03-2013.docx
100_postanovlenie_n50_13-03-2013.docx
http://ef-pmr.org/uploads/law/061_n34-ZID-V_27-04-2012.docx
http://ef-pmr.org/uploads/law/061_n34-ZID-V_27-04-2012.docx
061_n34-ZID-V_27-04-2012.docx
http://ef-pmr.org/uploads/law/109_postanovlenie_n168_06-07-2017.docx
http://ef-pmr.org/uploads/law/109_postanovlenie_n168_06-07-2017.docx
109_postanovlenie_n168_06-07-2017.docx
http://ef-pmr.org/uploads/law/085_zakon_n350-Z-III_4_11_2003.doc
http://ef-pmr.org/uploads/law/085_zakon_n350-Z-III_4_11_2003.doc
085_zakon_n350-Z-III_4_11_2003.doc
http://ef-pmr.org/uploads/law/123_prikaz_n19_14-04-2009.docx
http://ef-pmr.org/uploads/law/123_prikaz_n19_14-04-2009.docx
123_prikaz_n19_14-04-2009.docx
http://ef-pmr.org/uploads/law/116_postanovlenie_n50_13_03_2013.docx
http://ef-pmr.org/uploads/law/116_postanovle

http://ef-pmr.org/uploads/law/122_postanovlenie_n42_29-02-2016.rar
http://ef-pmr.org/uploads/law/122_postanovlenie_n42_29-02-2016.rar
122_postanovlenie_n42_29-02-2016.rar
http://ef-pmr.org/uploads/law/119_prikaz_n6423_16_05_2013.rar
http://ef-pmr.org/uploads/law/119_prikaz_n6423_16_05_2013.rar
119_prikaz_n6423_16_05_2013.rar
http://ef-pmr.org/uploads/law/053_postanovlenie_n295_04-12-2013.rar
http://ef-pmr.org/uploads/law/053_postanovlenie_n295_04-12-2013.rar
053_postanovlenie_n295_04-12-2013.rar
http://ef-pmr.org/uploads/law/052_prikaz_n978_28-09-2015.rar
http://ef-pmr.org/uploads/law/052_prikaz_n978_28-09-2015.rar
052_prikaz_n978_28-09-2015.rar
http://ef-pmr.org/uploads/law/136_prikaz_n312_10-04-2019.rar
http://ef-pmr.org/uploads/law/136_prikaz_n312_10-04-2019.rar
136_prikaz_n312_10-04-2019.rar
http://ef-pmr.org/uploads/law/022_zakon_o_soczoschite_chernob.rar
http://ef-pmr.org/uploads/law/022_zakon_o_soczoschite_chernob.rar
022_zakon_o_soczoschite_chernob.rar
http://ef-pmr.org/uploads


#### py069 Формируем HTML тест из DOC файла


In [ ]:
%%writefile phys.htm
<html>

<head>
<meta http-equiv=Content-Type content="text/html; charset=windows-1251">
<meta name=Generator content="Microsoft Word 11 (filtered)">
<title>Тестовые задания на внутривузовское тестирование </title>
<style>
<!--
 /* Font Definitions */
 @font-face
	{font-family:Tahoma;
	panose-1:2 11 6 4 3 5 4 4 2 4;}
@font-face
	{font-family:Calibri;
	panose-1:2 15 5 2 2 2 4 3 2 4;}
 /* Style Definitions */
 p.MsoNormal, li.MsoNormal, div.MsoNormal
	{margin:0cm;
	margin-bottom:.0001pt;
	font-size:12.0pt;
	font-family:"Times New Roman";}
h6
	{margin-top:12.0pt;
	margin-right:0cm;
	margin-bottom:3.0pt;
	margin-left:0cm;
	font-size:11.0pt;
	font-family:"Times New Roman";}
p.MsoAcetate, li.MsoAcetate, div.MsoAcetate
	{margin:0cm;
	margin-bottom:.0001pt;
	font-size:8.0pt;
	font-family:Tahoma;}
span.Heading6Char
	{font-family:"Times New Roman";
	color:windowtext;
	font-weight:bold;}
span.BalloonTextChar
	{font-family:Tahoma;
	color:windowtext;}
@page Section1
	{size:595.3pt 841.9pt;
	margin:2.0cm 42.5pt 2.0cm 3.0cm;}
div.Section1
	{page:Section1;}
-->
</style>

</head>

<body lang=RU>

<div class=Section1>

<p class=MsoNormal align=center style='text-align:center'><b>Тестовые задания
на внутривузовское тестирование </b></p>

<p class=MsoNormal align=center style='margin-left:5.65pt;text-align:center;
background:white'><b><span style='letter-spacing:-.25pt'>по дисциплине «Физика»</span></b></p>

<p class=MsoNormal align=center style='text-align:center'><b>для студентов </b><b><span
lang=EN-US>II</span><span lang=EN-US> </span>курса </b></p>

<p class=MsoNormal align=center style='text-align:center'><b>направления
подготовки <span style='letter-spacing:-.35pt'>09.03.04 «Программная инженерия»</span></b></p>

<p class=MsoNormal align=center style='margin-left:5.65pt;text-align:center;
background:white'><b><span style='letter-spacing:-.35pt'>профиля «Разработка
программно-информационных систем»</span></b></p>

<p class=MsoNormal align=center style='margin-left:5.65pt;text-align:center;
background:white'><b><span style='letter-spacing:-.35pt'>&nbsp;</span></b></p>

<p class=MsoNormal align=center style='text-align:center;text-indent:27.0pt;
background:white'><b>&nbsp;</b></p>

<p class=MsoNormal style='text-align:justify'>1.  Определите максимальную
энергию фотона в видимой серии спектра водорода (серии Бальмера).</p>

<p class=MsoNormal>а) 1,89 эВ   б) 3,41 эВ   в) 1,89 МэВ    г) 3,41 МэВ.</p>

<p class=MsoNormal>    </p>

<p class=MsoNormal style='text-align:justify'>2. Определите, на сколько
изменилась энергия электрона в атоме водорода при излучении атомом фотона с
длиной волны &#955;=4,86*10<sup>-7</sup> м.</p>

<p class=MsoNormal>а) 2,56 эВ   б) 5,12 эВ      в) 1,28 эВ   г) 10,24 эВ.</p>

<p class=MsoNormal>&nbsp;</p>

<p class=MsoNormal>3. Определите скорость электрона на третьей орбите атома
водорода.</p>

<p class=MsoNormal>а) 1,123 Мм/с    б) 1,123 м/с     в) 0,731 Мм/с   г) 0,731
м/с.</p>

<p class=MsoNormal>&nbsp;</p>

<p class=MsoNormal>4. Определите потенциал ионизации атома водорода.</p>

<p class=MsoNormal>а) 10,2 В    б) 13,6 В    в)  5,1 В   г)  27,2 В.</p>

<p class=MsoNormal>&nbsp;</p>

<p class=MsoNormal style='text-align:justify'>5. Определите длину волны де
Бройля для электрона, находящегося в атоме водорода на третьей боровской
орбите.</p>

<p class=MsoNormal>а) 4 нм     б) 3 нм   в) 2 нм   г) 1 нм.</p>

<p class=MsoNormal>&nbsp;</p>

<p class=MsoNormal style='text-align:justify'>6. Определите отношение
неопределенностей скорости электрона, если его координата установлена с
точностью до 10<sup>-5</sup> м, и пылинки массой 10<sup>-12</sup> кг, если ее
координата установлена с такой же точностью.</p>

<p class=MsoNormal>а) 2,2*10<sup>18</sup>   б) 2,2*10<sup>-18</sup>   в) 1,1*10<sup>-18</sup> 
г)  1,1*10<sup>18</sup>.</p>

<p class=MsoNormal style='text-align:justify'>&nbsp;</p>

<p class=MsoNormal style='text-align:justify'>7. Электрон с энергией 5эВ
движется в положительном направлении оси х, встречая на своем пути
прямоугольный потенциальный барьер высотой 10 эВ и шириной 0,1 нм. Определите
коэффициент прозрачности потенциального барьера.</p>

<p class=MsoNormal>а) 0,1  б) 0,2  в) 0,3   г) 0,4.</p>

<p class=MsoNormal>&nbsp;</p>

<p class=MsoNormal style='text-align:justify'>8. Определите наименьшую длину
волны рентгеновского излучения, если рентгеновская трубка работает при
напряжении 150 кВ.</p>

<p class=MsoNormal>а) 5,63 нм    б) 8,29 пм   в)  10,25 нм   г) 2,41 нм.</p>

<p class=MsoNormal>&nbsp;</p>

<p class=MsoNormal style='text-align:justify'>9.  Определите число свободных 
электронов, занимающих в среднем уровень энергии, равной энергии Ферми.</p>

<p class=MsoNormal>а) 4   б) 3  в) 2    г) 1</p>

<p class=MsoNormal>&nbsp;</p>

<p class=MsoNormal>10.  Определите массу нейтрального атома <img width=36 height=19 id="Рисунок 2" src="phys.files/image001.gif">.</p>

<p class=MsoNormal>а) 8,64*10<sup>-26</sup> кг    б)   8,64*10<sup>26</sup>кг   
в) 8,64*10<sup>-23</sup>кг   г) 8,64*10<sup>23</sup>кг.</p>

<p class=MsoNormal>&nbsp;</p>

<p class=MsoNormal style='text-align:justify'>11. Определите, пользуясь
таблицей Менделеева число нейтронов и протонов в атомах  урана.</p>

<p class=MsoNormal>а) 146    92      б)  92  146      в) 238    92     г) 92  
238</p>

<p class=MsoNormal>&nbsp;</p>

<p class=MsoNormal style='text-align:justify'>12. Определите энергию связи ядра
атома гелия <img
width=32 height=19 id="Рисунок 4" src="phys.files/image002.gif">. Масса
нейтрального атома гелия равна 6,6467*10<sup>-27</sup>кг.</p>

<p class=MsoNormal>а) 14,2 МэВ   б)  7,1 МэВ   в) 28,4 МэВ   г)  56,8 МэВ.</p>

<p class=MsoNormal>&nbsp;</p>

<p class=MsoNormal style='text-align:justify'>13. Определите, какая часть
начального количества ядер радиоактивного изотопа распадется за время, равное
двум периодам полураспада.</p>

<p class=MsoNormal>а) 0,75    б) 0,5   в) 0,25     г) 0,4.</p>

<p class=MsoNormal>&nbsp;</p>

<p class=MsoNormal style='text-align:justify'>14. Активность некоторого
радиоактивного изотопа в начальный момент времени составляла 100Бк. Определите
активность этого изотопа по истечении промежутка времени, равного половине
периода полураспада.</p>

<p class=MsoNormal>а) 70,7 Бк   б)  25,6 Бк    в) 30,8 Бк   г) 48,9 Бк.</p>

<p class=MsoNormal>&nbsp;</p>

<p class=MsoNormal>15. Пользуясь таблицей Менделеева и правилами смещения,
определите, в какой элемент превращается  после трех &#945;- и трех &#946;-
распадов.</p>

<p class=MsoNormal>а) <img
width=43 height=19 id="Рисунок 6" src="phys.files/image003.gif">   б) <img width=43 height=19 id="Рисунок 8" src="phys.files/image004.gif">   в) <img width=42 height=19 id="Рисунок 10" src="phys.files/image005.gif">  г) <img width=44 height=19 id="Рисунок 12" src="phys.files/image006.gif"></p>

<p class=MsoNormal align=center style='text-align:center;text-indent:27.0pt;
background:white'><b>&nbsp;</b></p>

</div>

</body>

</html>


In [4]:
%%writefile test_calc.js
function CalcMark(){
    forms = document.querySelectorAll( '.answers' );
    cnt_form = forms.length;
    res = "";
    for(num_form=0; num_form<cnt_form; num_form++){
        form =forms[num_form];
        sel_input = form.querySelector('input:checked');
        if( sel_input ){
            res += form.name + ":" + sel_input.value + ";";
        } else {
            res += form.name + ":0;";
        } 
     }
    document.formrez.test_res.value=res;
  }


Writing test_calc.js


In [7]:
#coding: cp1251
#form_test.py
import re

#--функция обработки одного блока вопросов    
def make_form(answ_block, form_num):
    ls_answ = []
    loc_pos = answ_block.find(')')
    while loc_pos >=0:
        loc_beg = loc_pos-1
        loc_pos = answ_block.find(')', loc_pos+1)
        loc_end = loc_pos-2 if loc_pos > 0 else len(answ_block)-4
        answ = answ_block[loc_beg:loc_end]
        ls_answ.append(answ)
    form = '\n<FORM NAME ="f'+ str(form_num) + '" class = "answers">'
    for answ_num, answ in enumerate(ls_answ):
        form +='\n    <INPUT TYPE="radio" NAME="rd1" value ='+str(answ_num+1) +'>'+answ + '<br>'
    form += '\n</FORM>'
    form +='\n<hr color = blue align = center width = 50%>'
    new_block = '<p>'+form+'\n</p>'
    return new_block


#--открыть исходный файл
fh = open('phys.htm')
text = fh.read()
fh.close()

#--удалить комментарии
rex = re.compile("<!--[^>]*?>")
text1 = re.sub(rex,"", text)

#--удалить тэг стиля
rex = re.compile("<style>[^>]*?>")
text2 = re.sub(rex,"", text1)

#--упростиь тэги параграфов
rex = re.compile("<p[^>]*?>")
text3 = re.sub(rex,"<p>", text2)

#--упростиь тэги span
rex = re.compile("<span[^>]*?>")
text = re.sub(rex,"<span>", text3)

#--общий цикл обработки страницы
new_page = ''
form_num = 1
pos_beg = text.find("<p>а)")
pos_end = 0
while pos_beg >=0:
    new_page += text[pos_end:pos_beg]
    pos_end = text.find("</p>", pos_beg) + 4
    answ_block = text[pos_beg:pos_end]
    new_block = make_form(answ_block, form_num)

    #-- вставка для контроля
    if form_num == 1:
        print (answ_block)
        print (new_block)

    new_page += new_block
    pos_beg = text.find("<p>а)", pos_end)
    form_num +=1

#--дописать подвал страницы

footer ='''
\n<script src="test_calc.js"></script>

<p></p>
   <FORM NAME ="formrez">
      <P  ALIGN = CENTER >
      <INPUT TYPE="text" NAME="test_res" id="test_res" SIZE=80 VALUE=" " style="font-size:20px">
      <br>     
      <INPUT TYPE="button" VALUE="Результат" onClick="CalcMark();" style="font-size:20px">
   </FORM>\n
'''
new_page += footer
new_page += text[pos_end:]


#--записать результат в файл
fh = open('phys4.htm','w')
fh.write(new_page)
fh.close()


<p>а) 1,89 эВ   б) 3,41 эВ   в) 1,89 МэВ    г) 3,41 МэВ.</p>
<p>
<FORM NAME ="f1" class = "answers">
    <INPUT TYPE="radio" NAME="rd1" value =1>а) 1,89 эВ  <br>
    <INPUT TYPE="radio" NAME="rd1" value =2>б) 3,41 эВ  <br>
    <INPUT TYPE="radio" NAME="rd1" value =3>в) 1,89 МэВ   <br>
    <INPUT TYPE="radio" NAME="rd1" value =4>г) 3,41 МэВ.<br>
</FORM>
<hr color = blue align = center width = 50%>
</p>


In [74]:
import re
text='''<DT><A HREF="http://www.zpmer.ru/index.php?option=com_salary&list_limitstart=140&view=firm_details_guest&employer=1183&Itemid=63" ADD_DATE="1438115355" ICON="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAABAAAAAQCAYAAAAf8/9hAAACsUlEQVQ4jXWTS2tkVRSF1zqPeyu3qm5V6tHaKBGcpBFBGhRE2kc7ycjH1KkjwYkTFZyLOHHiH/AHCNKSgRMhQutAfMyU5A+IqSRVqdTj1r3nnOUgsdMivX/A2mt9a28CwEf3J921M8TVuLRyrRVZBemLvSenJIVHjHv3/qQry98LY7cVoySRLOah76OJcfHEPl4CsHqkQNsZQhoba0sYg9DUa8bN7Wd//hotVHjvuy9d/81RF6cAhoDm1I2DyeKBwDIktSybGIMoQcasXv32s/ULv3zzk3F+u4ZxIWwTPclGMhS4OH5tcuvGARYA4P5vili0H8uizcbG+R5jhDGABFgDxAjLckTg5DqCUvLWWlJCiNHXIRAShEt2MUmAFBMIIZw+zGAZkrYyNwHYAIB1fnrr5s3KERMKEYAFUAKkQILgYH6ia78A9z749PFMjY+hYd0E/fXj/uLtomqXmYvvFCt4n/9BoCQAAefWzXa+et+mD/f+XnE0GnVHg+3fBHpJiaQBaUCjJqF+I6vufPKUPXKGJQHUjc5/vbN87uiZ7F6+nt11rVbL07oixfiyz7JZjJGr1SqVmQZy7R9aYm4TjpMQCMAknPnGNSB6KXSMm8/n6HQ6m+l0ejyZXPc7Ho+b4bAd98/rxd3b9hXPaAAgNtSfz0fH2uDCLZLr9/siafI8z/5lAkDDYZYrMT39euocvWi+F+wgRQkAbMoyMNVt2ehmsxmLokibzaa+ZHTZnbSpJSXvczKlkTXsxyt3hkAETtebM7myLAEgL4pivLOz41NKNMZIwjaAHKgAZP89NQIg/VY+oKuqqul2u+uiKA6uWqAkGUMTAirYphay05SUpMsIKYIApuvNmQgAu7u73eVy6R/e0usBIbSbw8PDxef3Rp2ti/Tg3TEA1o3Rx2+dXPwDuF9YClrb1zsAAAAASUVORK5CYII=">ЗПмер - зарплата в Yota, статистика зарплат по Санкт-Петербург, средняя зарплата Санкт-Петербург</A>'''
# очистить параметры тэга 
text="<p aaa>bbbb</p>"
tag = "p"
st = "<"+tag+"[^>]*?>"
rex = re.compile(st)
print(re.sub(rex, "<"+tag+">",text))

<p>bbbb</p>


In [75]:
# извлечь содержимое тега
text='''<A aaa>safsdf href = "sdfsfsfsfsfs" sfasfsaf</A>'''
tag = "A"
def tag_content(tag, text):
    st = "<"+tag+"[^>]*?>([\s\S]*?)<\/"+tag+">"
    rex = re.compile(st)
    res = re.search( rex, text)
    return res.group(1)
print(tag_content(tag, text))

safsdf href = "sdfsfsfsfsfs" sfasfsaf


In [86]:
# извлечь ссылку 
def get_a(text):
    tag = "A"
    st = "(HREF=\""+"[^\"]*?"+"\")"
    rex = re.compile(st)
    res = re.search( rex, text)
    return "<a>"+res.group(1)+"</a>"
text="safsdf HREF=\"sdfsfsfsfsfs\" sfasfsaf"
print(text)
print(get_a(text))

safsdf HREF="sdfsfsfsfsfs" sfasfsaf
<a>HREF="sdfsfsfsfsfs"</a>


In [94]:
# извлечь ссылку 
def tag_content(tag, text):
    st = "<"+tag+"[^>]*?>([\s\S]*?)<\/"+tag+">"
    st = "<"+tag+"[^>]*?>([\s\S]*?)<\/"+tag+">"
    rex = re.compile(st)
    res = re.search( rex, text)
    return res.group(1)

def get_a(text):
    tag = "A"
    text = tag_content(tag, text)
    st = "(HREF=\""+"[^\"]*?"+"\")"
    rex = re.compile(st)
    res = re.search( rex, text)
    return "<a>"+res.group(1)+"</a>"
text='''<A aaa>safsdf HREF="sdfsfsfsfsfs" sfasfsaf</A>'''
print(get_a(text))

<a>HREF="sdfsfsfsfsfs"</a>


In [100]:
# извлечь ссылку 
t='''<A HREF="https://www.yandex.ru/" ADD_DATE="1470040980" ICON="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAABAAAAAQCAYAAAAf8/9hAAABiUlEQVQ4jX3SvWqUURSF4Wefb+IUxkIxXXD8GZVgYWtrFRsrGxuvQQsvQPAOIta2WiiCQjqtLEXBNoKGgE1IAgqazMzZFjMT/SZDdnV+1noXZ58dZiqJIHPZ5dr1OMKdKDoq0raBK7Fpd6rrzDX3rGTXekm9rF4Z+qK4hyU/1dnQFiCJesm7vCqH5909vOt7nX0HyampFspsunNWoriZQx863zxPStJBF+nM2Dit8t96fNHoKzL4NEkpwRCZ0TbPB4QOso4MWsqQR9wzgAQjP6RSGr0Yn0XSSEVONPMAYdzdre8+Z/VVYzUvuB4MglFygGKnDZn3jb+zeqB4qVjPS57VsBX0se+0Ynf23Uchmctu6Xqr0ah+4SR2bOvHnr2prswCDqELFnUwsmbDEt5j0V57kI4Agpqc0HikGtr3JPiDAyGcbfegBcjJfnTRbY1rqhexZeNQmVJtP7vVxEl6yfBQVQ2stfjHDdI0Xc9qdNxQvYlNH/OfZt4ctQcpKRbcl3I48PQ447T+AlRml3CFHoc8AAAAAElFTkSuQmCC">Яндекс</A>'''


def tag_content(tag, text):
    st = "<"+tag+"[^>]*?>([\s\S]*?)<\/"+tag+">"
    st = "<"+tag+"[^>]*?>([\s\S]*?)<\/"+tag+">"
    rex = re.compile(st)
    res = re.search( rex, text)
    return res.group(1)

def get_tagparams( tag, text ):
    st = "<" + tag + "([^>]*?)>"
    rex = re.compile(st)
    res = re.search( rex, text)
    return res.group(1)


def get_a(text):
    tag = "A"
    text = get_tagparams(tag, text)
    st = "(HREF=\""+"[^\"]*?"+"\")"
    rex = re.compile(st)
    res = re.search( rex, text)
    return "<a>"+res.group(1)+"</a>"
text='''<A aaa>safsdf HREF="sdfsfsfsfsfs" sfasfsaf</A>'''
print(get_a(t))

<a>HREF="https://www.yandex.ru/"</a>


#### py060 Простая отладка сетевых приложений
#### py061 Сетевой вариант крестиков-ноликов на Питоне
#### py062 элементы тестирования консольной программы
#### py063 игра Змейка
#### py064 Python отладчик для исправления ошибок
#### py065 Python работа с сетью на канальном уровне
#### py066 Пятнашки на Питоне
#### py067 Питон программа управления ШИМ контроллера
#### py068 Питон скачиваем документы по ссылкам
#### py069 Формируем HTML тест из DOC файла


[abglazov](http://abglazov.rfpgu.ru/)  
[py060 Простая отладка сетевых приложений](https://www.youtube.com/watch?v=UpkYtsiYRks&list=PLku9se_HAVOp8pSgrIWGIjHLP-EOw_ywq&index=25)  
[]()  
[]()  
